<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Рекуррентные нейросети](#toc1_)    
  - [Одномерная рекуррентная сеть](#toc1_1_)    
    - [Подробнее про gradient clipping](#toc1_1_1_)    
  - [Долгосрочная-краткосрочная память, Long short-term memory, LSTM](#toc1_2_)    
  - [Управляемые рекуррентные нейроны, Gated recurrent units, GRU](#toc1_3_)    
  - [Simple recurrent unit](#toc1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Рекуррентные нейросети](#toc0_)

Рабочая лошадка нейросетевых моделей для обработки **последовательностей** — данных вообще и текста в частности.

Основная идея в том, что у нас есть некоторое внутреннее состояние, которое мы обновляем после прочтения очередного элемента входной последовательности. И по идее, это внутреннее состояние должно содержать в себе информацию обо всём прочитанном ранее тексте. 

В свёртках же — наоборот, мы состояние не хранили, а обрабатывали кусок текста определённого размера — сразу. 

Рекуррентные нейросети могут использоваться как для предсказания одной величины по тексту (например, для классификации), так и для предсказания каких-либо величин для каждого элемента входной последовательности. 

**Суть алгоритма обучения**

На кажом шаге:

1. прочитать и обработать очередной элемент входной последовательности
   $$z_t=W_{input} \cdot x_t: W_{input} \in \R^{d_{hidden} \times d_{input}}$$
   
2. вычислить новое значение ячейки памяти, исходя из ее старого значения и входного элемента
   $$h_t=\tanh (W_{hidden}\cdot h_{t-1} + z_t): W_{hidden} \in \R^{d_{hidden} \times d_{hidden}}$$

3. вычислить выход
   $$y_t = W_{outout}\cdot h_t$$


$W_{input}$ - матрица весов, позволяющая получить векторное представление входных данных, то есть, эмбеддинг  
$W_{hidden}$ - матрица, транслирующая данные из пространства эмбеддингов в пространство внутреннего представления данных в модели  
$W_{output}$ - матрица, предназначенная для решения конечной задачи, например, она может переводить внутреннее состояние модели в набор предсказаний классов для токенов или текста целиком.

                              y(3) -> Loss
                              ^
                              |
      h(0) -> h(1) -> h(2) -> h(3)
              ^       ^       ^
              |       |       |
              z(1)    z(2)    z(3)

Итак, мы читаем входной текст слово за словом или символ за символом и, сначала, обрабатываем его (например, мы можем применить к нему линейное преобразование).  
Затем мы вычисляем новое значение состояния. В простейшем случае оно зависит от значения состояния на предыдущем шаге и от текущего входа.  
Далее мы делаем предсказание, используя только текущее значение состояния, а потом читаем следующее слово и повторяем всю процедуру заново. 

Преимущество:
- потенциально RNN гораздо мощнее, чем CNN

Недостатки:
- скрытое состояние зависит от предыдущего, и мы обрабатываем элемент за элементом последовательно, поэтому RNN не так хорошо используют возможности параллельных вычислений в графических ускорителях и, в итоге, медленнее CNN
- есть проблемы сходимости (**затухание и взрыв градиента**)
- необходимость поиска баланса между мощностью и простотой обучения



## <a id='toc1_1_'></a>[Одномерная рекуррентная сеть](#toc0_)

Классическая модель (vanilla run)

Пусть размерность всех переменых (входные данные и скрытого состояния) равна 1, т.е. работаем только со скалярными данными. И для простоты считаем, что нужно оценить одну величину по всему тексту - например, класс текста на основе значения состояния на последнем шаге, т.е. после прочтения всего текста.

- Вход: $z_t \in \R$
- Скрытое состояние: $h_t = f(w\cdot h_{t-1} + z_t)$, где $f$ - функция активации, $w \in \R$ - параметр функции перехода
- Выход: $y_t = g(h_t)$
- Функционал качества: $Loss(y_t)$

Исходя из $Loss(y_t)$ будем **градиентным спуском** настраивать параметры нейросети. 

Здесь не важно, какую конкретно функцию активацию мы используем. 

Последовательность вычислений:

- $h_0 \in \R$
- $h_1 = f(w\cdot h_0 + z_1)$
- $h_2 = f(w\cdot h_1 + z_2) = f(w\cdot f(w\cdot h_0 + z_1) + z_2)$
- etc.

Получаем глубокую композицию функций. Это **прямой проход по сети**.

**Обратный проход**

$Loss(y_t) =  Loss(g(h_t)) = Loss(g(f(w\cdot h_{t-1} + z_t))) = Loss(g(f(w\cdot f(w\cdot h_{t-2} + z_{t-1}))) + z_t))) = ...$

- предсказание ($y$) — это функция от последнего состояния, которое, в свою очередь, является функцией предпоследнего состояния и последнего слова, и так далее...
  
Найдем градиент по весам (направление наибольшего роста функции потерь в зависимости от весов):


- $y$ это сложная функция, поэтому применяем правило цепочки
- значение ошибки $Loss(g(f(x; w)))$, где $x$ - входные данные, $w$ - параметры модели, $Loss$, $g$, $f$ - некоторые функции.

$$\frac{\partial Loss(y_t)}{\partial w} = \frac{\partial Loss(y_t)}{\partial w} \frac{\partial g}{\partial f} \frac{\partial f}{\partial w} = \frac{\partial Loss(y_t)}{\partial w} \frac{\partial g}{\partial f} \frac{\partial f(w\cdot h_{t-1} + z_t)}{\partial w} \\
\frac{\partial f(w\cdot h_{t-1} + z_t)}{\partial w} = f'(w\cdot h_{t-1} + z_t)\cdot (w\cdot h_{t-1} + z_t)'= f'_t \cdot (w\cdot h_{t-1})' = f'_t\cdot(w'\cdot h_{t-1} + w\cdot h'_{t-1}) = \\
= f'_t\cdot(h_{t-1} + w\cdot f(w\cdot h_{t-2} + z_{t-1})') = f'_t\cdot(h_{t-1} + w\cdot f'_{t-1} \cdot f(w\cdot h_{t-3} + z_{t-2})') = \\
= \sum_{i=1}^t (h_{i-1} \cdot w^{t-i} \prod_{j=1}^t f'_j)$$

- производная функции потерь по весам (параметрам модели) $w$ зависит от **всех шагов**, а также включает произведение всех **производных функции активации** на нескольких шагах — вот тут-то собака и зарыта.

**Пример:**

Допустим, $w=1.1$, тогда за 100 шагов в градиенте функции потерь накопится множитель $w^{99} = 1.1^{99} = 12528$

Така ситуация и называется **взрыв градиента**.

- если $f = \tanh$, $0 < \tanh' < 1 \rightarrow$ **затухание градента**. Информация с первых шагов почти никак не учитывается при вычислении обновления весов. И в этом случае весь смысл использования рекуррентности исчезает.
- если $|w \cdot f'| > 1 \rightarrow$ **взрыв градента**. Приводит к переполнению и катастрофическому падению точности вычислений.
- одна из задач построения рекуррентной сети - обеспечить баланс, недопуская ни того, ни другого (а соответствующие эффекты действуют одновременно друг против друга - у нас их произведение в формуле градиента ошибки)

Со **взрывом градиента** борются очень просто. Один из вариантов — сначала честно считают градиентные шаги для всех параметров, а потом, если какой-то градиент по модулю превышает некоторый порог, то он заменяется на значение порога со знаком (gradient clipping). То есть слишком большие градиенты просто обрезаются.[1,2,3] Бороться с **затуханием градиента** гораздо сложнее — этому посвящено множество работ. Давайте рассмотрим парочку.

[1] https://machinelearningmastery.com/how-to-avoid-exploding-gradients-in-neural-networks-with-gradient-clipping/  
[2] http://www.wildml.com/deep-learning-glossary/  
[3] How to Avoid Exploding Gradients With Gradient Clipping https://machinelearningmastery.com/how-to-avoid-exploding-gradients-in-neural-networks-with-gradient-clipping/  

### <a id='toc1_1_1_'></a>[Подробнее про gradient clipping](#toc0_)

Градиент - это вектор направления. Если его клипать ровно как говорится в видео (то есть каждый элемент вектора независимо обрезать, как это делает, например, `tensor.clamp` в `pytorch`), то после клиппинга вектор будет указывать в другом направлении и процесс обучения может **перестать сходиться**. Например, если изначально градиент был [0.1, 100], то после клиппинга с порогом 1 он станет [0.1, 1], это вообще не в ту сторону, даже не близко от направления наибольшего градиента.

В PyTorch есть две функции для gradient clipping:.

`clip_grad_value_` - заменяет значение градиента на порог, если значение превосходит порог по модулю. При этом направление вектора меняется.

`clip_grad_norm_` - масштабирует весь вектор градиента так, чтобы его норма (считай длина) не превышала заданный порог. При этом направление вектора градиента не меняется.

Что лучше использовать - надо пробовать в каждом конкретном случае. `clip_grad_norm_` более корректный математически, но может приводить к тому, что обучение будет слишком медленным, а `clip_grad_value_` может приводить к расхождению процесса обучения (далеко не всегда), но не замедляет этот процесс.

## <a id='toc1_2_'></a>[Долгосрочная-краткосрочная память, Long short-term memory, LSTM](#toc0_)

- самый часто используемый вид рекурренток. 

Придуман в конце девяностых годов[1].


<img src="./img/lstm.png" width="900">


$$i_t = \sigma (W_{ii}x_t + W_{hi}h_{t-1} + b_i) \\
f_t = \sigma (W_{if}x_t + W_{hf}h_{t-1} + b_f) \\
g_t = \tanh (W_{ig}x_t + W_{hg}h_{t-1} + b_g) \\
o_t = \sigma (W_{io}x_t + W_{ho}h_{t-1} + b_o) \\
c_t = f_t * c_{(t-1)} + i_t * g_t \\
h_t = o_t * \tanh(c_t) \\
i_t, f_t, o_t, c_t, h_t \in \R^d
$$
На первый взгляд, это очень сложная нейросеть (такая и есть). Но в её основе лежит простая и понятная идея. 

Цель — сохранение **объёма потока ошибки**.[2] 

- Предположим, что у нас есть труба, и через неё текут градиенты. И мы хотим, чтобы площадь сечения этой трубы была примерно одинаковой, чтобы у нас не было узких горлышек и очень широких областей. 
- Для этого была придумана так называемая "карусель постоянного объёма ошибки".[2] 

Это рекуррентность особого вида - **поток ошибки постоянного объема**. 

$$c_t = f_t * c_{(t-1)} + i_t * g_t \\ -\infin < c_t < \infin$$

- самая главная фишка —  при переходе от предыдущего значения к текущему мы не используем функцию активации, а вместо этого используем **операцию сложения**

На значение вектора "c" влияет его предыдущее состояние и ещё 3 сущности: 

- направление изменения внутреннего состояния вектора "c" относительно его предыдущего значения ($\tanh$ дает диапазон [-1, 1]):
$$g_t = \tanh (W_{ig}x_t + W_{hg}h_{t-1} + b_g) \\ -1 < g_t < 1$$

- "f" и "i", они отвечают за **амплитуду** изменения. 
  - Вектор "i" отвечает за **чувствительность к "g"** (он может ослабить влияние "g" на изменение вектора "c". Вектор "i" ещё называют входным шлюзом или "**input gate**"):  
$i_t = \sigma (W_{ii}x_t + W_{hi}h_{t-1} + b_i) \\ 0 < i_t < 1$
  - Вектор "f" отвечает за **чувствительность к предыдущему значению рекуррентного состояния** (ещё называют "шлюзом забывания" или "**forgetting gate**" — он позволяет совершать резкие изменения скрытого состояния и игнорировать всё, что было до определённого момента):  
$f_t = \sigma (W_{if}x_t + W_{hf}h_{t-1} + b_f) \\ 0 < f_t < 1$

- выход из рекуррентного состояния (всего в LSTM два рекуррентных вектора — "c" и "h"):
  - чтобы повысить мощность данной нейросети, к вектору "c" мы применяем нелинейность, чтобы получилось ещё одно рекуррентное состояние — на этот раз с более сложным преобразованием. 
$o_t = \sigma (W_{io}x_t + W_{ho}h_{t-1} + b_o), 0 < o_t < 1 \\
h_t = o_t * \tanh(c_t), -1 < h_t < 1 $

Здесь применяется всё та же идея "**гейтинга**", когда вектор значений умножается на вектор шлюза, который управляет потоком или амплитудой значений, но не может изменить знак — это один из ключевых приёмов в рекуррентках и в глубоких нейросетях в целом. 

**Количество параметров**

Если размер скрытого состояния — $d$, то количество параметров сети имеет порядок:

$$ 8 d^2 + 4 d $$

- это очень много. Как результат, LSTM **часто переобучаются** и плохо обобщаются на новые данные (но не всегда).

[1] Long Short-Term Memory, Sepp Hochreiter and Jürgen Schmidhuber, Neural Computation 1997 9:8, 1735-1780  
[2] Constant Error Carousel https://deepai.org/machine-learning-glossary-and-terms/constant%20error%20carousel

**Q**:

Какие методы регуляризации применяются в рекурентных сетях? Особенно в LSTM м GRU? Используют ли батч-нормализация и дропаут?

**A**:

Можно использовать рекуррентный dropout. Идея следующая: давайте будем применять dropout только к "нерекуррентным" связям (вертикальные связи на картинках). Таким образом, мы не ломаем механизм памяти, дропнутые связи в вертикальном направлении могут быть восстановлены в аналогичной параллельной вертикальной связи, там это они не дропнулись, т.к. по горизонтали все идет строго без дропов.

Батч-нормализацию (**пакетная нормализация** - нормализция входов (минус м.о. делить на с.к. по всей выборке), дает равноправие градиентых координат, более быстрый спуск) в рекуррентках непонятно как считать, обычно используют LayerNorm (мы используем его в семинаре про трансформер, но подробно не разбираем). 

\*) **NB**: нормализация меняет представление сети о данных в слое, поэтому еще вводятся параметры сжатия и сдвига для каждого входа, которые настраиваются в процессе обучения вместе с остальными параметрами модели (учитываются в формуле градиента функции потерь).

## <a id='toc1_3_'></a>[Управляемые рекуррентные нейроны, Gated recurrent units, GRU](#toc0_)

Упрощение модели LSTM

<img src="./img/gru.png" width="1000">



$$r_t = \sigma (W_{ir}x_t + W_{hr}h_{t-1} + b_r) \\
z_t = \sigma (W_{iz}x_t + W_{hz}h_{t-1} + b_z) \\
g_t = \tanh (W_{in}x_t + b_{in} + r_t*(W_{hn}h_{(t-1)} + b_{hn})) \\
h_t = z_t * h_{(t-1)} + (1 - z_t) * g_t \\
r_t, z_t, g_t, h_t \in \R^d
$$

Цель — сохранение **объёма потока ошибки**.

Однако теперь потоком ошибки управляют не два шлюза, а один — по сути, этот шлюз на каждом шаге осуществляет выбор между двумя альтернативами: оставить предыдущее значение, или обновить.

- это также рекуррентность особого вида - **поток ошибки постоянного объема**. 

$$h_t = z_t * h_{(t-1)} + (1 - z_t) * g_t \\ -\infin < h_t < \infin$$

- выбор оставить предыдущее значение, или обновить:
$$z_t = \sigma (W_{iz}x_t + W_{hz}h_{t-1} + b_z), \\ 0 < z_t < 1$$

- направление изменения внутреннего состояния вектора "h" относительно его предыдущего значения ($\tanh$ дает диапазон [-1, 1]):
  
$$g_t = \tanh (W_{in}x_t + b_{in} + r_t*(W_{hn}h_{(t-1)} + b_{hn}))\\ -1 < g_t < 1$$

- вектор "r" отвечает за **чувствительность памяти"** (чувствительность к предыдущему состоянию):  

$$r_t = \sigma (W_{ir}x_t + W_{hr}h_{t-1} + b_r) \\ 0 < r_t < 1$$

В результате, количество параметров **сократилось на треть** — ну что ж неплохо:

$$6 d^2 + 6d$$

На практике GRU и LSTM, в большинстве задач, работают практически одинаково и дают очень близкое качество. То есть сеть упростилась, стала учиться лучше, но при этом осталось достаточно мощной. 

Однако есть вторая проблема — скорость. Каждый элемент вектора скрытого состояния по-прежнему зависит от всего вектора предыдущего состояния. 

## <a id='toc1_4_'></a>[Simple recurrent unit](#toc0_)

Дальнейшее упрощение привело к модели[1], где заменили матрицу и матричное произведение на вектор на **поэлементное произведение** (тут \*). 

$$f_t = \sigma (W_{ir}x_t + v_f * с_{(t-1)} + b_f) \\
r_t = \sigma (W_{iz}x_t + v_f * с_{(t-1)} + b_r) \\
c_t = f_t * c_{(t-1)} + (1 - f_t) * (W_{c}x_t)) \\
h_t = r_t * c_t + (1 - r_t) * x_t \\
f_t, r_t, c_t, h_t \in \R^d$$


- элементы векторов состояния "с" зависят только от тех же элементов с предыдущего шага

$$c_t = \{ c_t^i\}, c_t^i = Q(c_{t-1}^i, x_t), c_t^i \perp c_{t-1}^i, \forall i, j : i \neq j$$

- значения $c_t^i$ и $h_t^i$ можно параллельно вычислять (ускорение в 5-9 раз по ср. с LSTM)

- количество параметров еще в 2 раза меньше: $3d^2 + 4d$

С учётом того, что, на практике, размерность этих векторов составляет от нескольких сотен до пары тысяч, это даёт очень хороший прирост производительности. 

Кроме того, в этой сети ещё в два раза меньше параметров, поэтому она ещё меньше переобучается (хотя и слабее, чем LSTM), но, на практике, за счёт более простой структуры, процесс обучения идёт более эффективно и качество решения задачи остаётся прежним, или может даже немного улучшится. 

Итого, самая старая, проверенная и популярная — LSTM. Относительно недавно был предложен облегченный вариант — "грушка". И ещё более недавно придумали, как можно упростить и, при этом, ускорить рекуррентки — примером такой работы является simple recurrent unit.

[1] Lei T. et al. Simple recurrent units for highly parallelizable recurrence //arXiv preprint arXiv:1709.02755. – 2017. (https://arxiv.org/abs/1709.02755)  




**Например**

Скрытое состояние вычисляется по формуле $h_{t} = v * h_{t-1}$, где $h_{t}, h_{t-1} \in \mathbb{R^d}$ - новое состояние и предыдущее, а $v \in \mathbb{R^d}$ - вектор перехода, $v * h$ - операция поэлементного произведения двух векторов.

Тогда компоненты вектора $h_{t-1}$, от которых зависит третья компонента вектора $h_{t}[3]$, это только $h_{t-1}[3]$, а не весь вектор (очень глубокое замечание).